# load thru drive (still buggy)

In [ ]:
# imports
import numpy as np
import os
from matplotlib import pyplot as plt

from datetime import date


import pandas
import sqlalchemy
from sqlalchemy.orm import sessionmaker

In [ ]:
%matplotlib notebook

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
%cd '/content/drive/My Drive'

/content/drive/My Drive


In [ ]:
mhw_file = '/content/drive/My Drive/MHW/db/mhws_allsky_defaults.db'

In [ ]:
engine = sqlalchemy.create_engine('sqlite:///'+mhw_file)

In [ ]:
connection = engine.connect()
connection

In [ ]:
engine.has_table("MHW_Events")

True

In [ ]:
pd_tbl = pandas.read_sql('SELECT date, lat, lon FROM MHW_Events;', connection)

OperationalError: ignored

In [ ]:
pd_tbl

In [ ]:
metadata = sqlalchemy.MetaData()
mhw_tbl = sqlalchemy.Table('MHW_Events', metadata, autoload=True, autoload_with=engine)

In [ ]:
from datetime import datetime
query = sqlalchemy.select([mhw_tbl]).where(sqlalchemy.and_(
    # range of lat 
    sqlalchemy.between(
        # pass functions?
        mhw_tbl.columns.lat, 30.0, 35.0
    ),
    # and range of lon
    sqlalchemy.between(
        # pass functions?
        mhw_tbl.columns.lon, 30.0, 35.0
    ),
    # and range of dates
    sqlalchemy.between(
        # pass functions?
        mhw_tbl.columns.date, datetime.strptime('Jun 1 2005', '%b %d %Y'), datetime.strptime('Jun 1 2006', '%b %d %Y')
    )
    ))

In [ ]:
query2 = sqlalchemy.select([mhw_tbl]).where(sqlalchemy.and_(
    mhw_tbl.columns.ievent == 108, mhw_tbl.columns.time_start == 737341, mhw_tbl.columns.duration == 14))

In [ ]:
query3 = "SELECT lat FROM MHW_Events WHERE time_start=737341"  # String containing the SQL query to select all rows
df = pandas.read_sql_query(query3, engine)  # Finally, importing the data into DataFrame df
df.info()

In [ ]:
result2 = connection.execute(query2)

In [ ]:
print()
for row in result2:
    print(row)


(43983, 737341, 737354, 737343, 14, 14, 0, 0, 0, 0, 0, 108, datetime.datetime(2019, 10, 9, 0, 0), 15.375, 84.625, 1.3529129028320312, 1.0404736995697021, 0.19957762956619263, 14.566631317138672, 0.6140308380126953, 30.35999870300293, 0.30911731719970703, 30.022144317626953, 0.1935850828886032, 0.21883374452590942, 4.327642440795898, 420.3100280761719, 0.24236640334129333, 0.05813167244195938)


OperationalError: ignored

In [ ]:
result = connection.execute(query)

In [ ]:
flag = True
while flag:
    partial_results = result.fetchmany(50)
    if(partial_results == []): 
    	flag = False
    # convert to df
result.close()

OperationalError: ignored

In [ ]:
from IPython.display import display
import pandas as pd
import sqlalchemy

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    #sql(query)

display_results(query)

OperationalError: ignored

In [ ]:
pd_tbl = pandas.read_sql_table('MHW_Events', con=engine, 
                               columns=['date', 'lon', 'lat', 'duration' , 
                                        'ievent', 'time_start', 'index'],
                              index_col=0)

OperationalError: ignored

In [ ]:
from google import colab

In [ ]:
colab.NotebookNotary.db_file = u':memory:'

AttributeError: ignored

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
pandas.read_sql(session.query(mhw_tbl).filter(mhw_tbl.columns.lat == 100.0).statement,session.bind) 

OperationalError: ignored

###Experiment with Other dataset

In [ ]:
from sqlalchemy import create_engine
#engine = create_engine('sqlite:///example.db', echo=True)
engine = create_engine('sqlite:///:memory:', echo=True)
#engine = create_engine('sqlite:///:memory:')
conn = engine.connect()

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

2020-06-23 21:44:28,690 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-06-23 21:44:28,692 INFO sqlalchemy.engine.base.Engine ()
2020-06-23 21:44:28,700 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-06-23 21:44:28,703 INFO sqlalchemy.engine.base.Engine ()


In [ ]:
from IPython.display import display
import pandas as pd
import sqlalchemy

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    #sql(query)

In [ ]:
!pip install sqlalchemy_explore

  Created wheel for sqlalchemy-explore: filename=sqlalchemy_explore-0.1.2-cp36-none-any.whl size=4544 sha256=a72c3946033ba440aee14944e75033dabb1790ae4df277caf6827d6934a889c0
  Stored in directory: /root/.cache/pip/wheels/af/7d/67/1c55901ebff236da60829800b9a2ebfa8b52c455db565a1acd
Successfully built sqlalchemy-explore


In [ ]:
from sqlalchemy.ext.declarative import declarative_base
import sqlalchemy_explore

### the basic base class for SQLAlchemy schema objects
# Base = declarative_base(bind=engine)

### base class including utils like an __repr__ method
### see https://pypi.org/project/sqlalchemy-explore/
Base = declarative_base(cls=sqlalchemy_explore.ReflectiveMixin)

In [ ]:
from sqlalchemy import Column, DateTime, ForeignKey, Integer, NVARCHAR, Numeric, Sequence
from sqlalchemy.orm import relationship

class Customer(Base):
    __tablename__ = 'customers'

    CustomerId = Column(Integer, Sequence('customer_id_seq'), primary_key=True)
    FirstName = Column(NVARCHAR(40), nullable=False)
    LastName = Column(NVARCHAR(20), nullable=False)
    Company = Column(NVARCHAR(80))
    Address = Column(NVARCHAR(70))
    Phone = Column(NVARCHAR(24))
    Email = Column(NVARCHAR(60), nullable=False)
    
class Item(Base):
    __tablename__ = 'items'
    
    ItemId = Column(Integer, Sequence('item_id_seq'), primary_key=True)
    Name = Column(NVARCHAR(40), nullable=False)
    Price = Column(Numeric, nullable=False)

class Purchase(Base):
    __tablename__ = 'purchases'
    
    PurchaseId = Column(Integer, Sequence('purchase_id_seq'), primary_key=True)
    ItemId = Column(ForeignKey('items.ItemId'), nullable=False, index=True)
    CustomerId = Column(ForeignKey('customers.CustomerId'), nullable=False, index=True)
    Date = Column(DateTime, nullable=False)
    
    item = relationship('Item')
    customer = relationship('Customer')

# load locally (working)

Download DB and load locally following these instructions:

https://stackoverflow.com/questions/61030755/connect-to-postresql-database-from-google-colab#:~:text=Connect%20to%20the%20local%20runtime,connected%20to%20your%20local%20runtime.

https://research.google.com/colaboratory/local-runtimes.html

In [1]:
mhw_file = '/Users/kamil/Downloads/mhws_allsky_defaults.db'

In [4]:
!pip3 install pandas
!pip3 install sqlalchemy

     |████████████████████████████████| 10.0MB 2.5MB/s 


In [5]:
# imports
import numpy as np
import os
from matplotlib import pyplot as plt

from datetime import date


import pandas
import sqlalchemy
from sqlalchemy.orm import sessionmaker

In [6]:
engine = sqlalchemy.create_engine('sqlite:///'+mhw_file)

In [7]:
engine.has_table("MHW_Events")

True

In [8]:
metadata = sqlalchemy.MetaData()
mhw_tbl = sqlalchemy.Table('MHW_Events', metadata, autoload=True, autoload_with=engine)

In [9]:
mhw_tbl

Table('MHW_Events', MetaData(bind=None), Column('index', BIGINT(), table=<MHW_Events>), Column('time_start', INTEGER(), table=<MHW_Events>), Column('time_end', INTEGER(), table=<MHW_Events>), Column('time_peak', INTEGER(), table=<MHW_Events>), Column('duration', INTEGER(), table=<MHW_Events>), Column('duration_moderate', INTEGER(), table=<MHW_Events>), Column('duration_strong', INTEGER(), table=<MHW_Events>), Column('duration_severe', INTEGER(), table=<MHW_Events>), Column('duration_extreme', INTEGER(), table=<MHW_Events>), Column('category', INTEGER(), table=<MHW_Events>), Column('n_events', INTEGER(), table=<MHW_Events>), Column('ievent', BIGINT(), table=<MHW_Events>), Column('date', DATETIME(), table=<MHW_Events>), Column('lat', FLOAT(), table=<MHW_Events>), Column('lon', FLOAT(), table=<MHW_Events>), Column('intensity_max', FLOAT(), table=<MHW_Events>), Column('intensity_mean', FLOAT(), table=<MHW_Events>), Column('intensity_var', FLOAT(), table=<MHW_Events>), Column('intensity_cum

In [10]:
from datetime import datetime
query = sqlalchemy.select([mhw_tbl]).where(sqlalchemy.and_(
    # range of lat 
    sqlalchemy.between(
        # pass functions?
        mhw_tbl.columns.lat, 30.0, 35.0
    ),
    # and range of lon
    sqlalchemy.between(
        # pass functions?
        mhw_tbl.columns.lon, 30.0, 35.0
    ),
    # and range of dates
    sqlalchemy.between(
        # pass functions?
        mhw_tbl.columns.date, datetime.strptime('Jun 1 2005', '%b %d %Y'), datetime.strptime('Jun 1 2006', '%b %d %Y')
    )
    ))

In [12]:
connection = engine.connect()
connection

In [13]:
result = connection.execute(query)

In [14]:
result_set = result.fetchall()

In [15]:
result_set[:3]

[(24022, 732152, 732156, 732154, 5, 5, 0, 0, 0, 0, 0, 24, datetime.datetime(2005, 7, 25, 0, 0), 31.375, 30.125, 1.291748046875, 1.1371936798095703, 0.1312417984008789, 5.685968399047852, 0.4629039764404297, 27.799999237060547, 0.30727043747901917, 27.644001007080078, 0.13109475374221802, 0.12289836257696152, 1.5363521575927734, 138.22000122070312, 0.15309639275074005, 0.22540283203125),
 (24023, 732372, 732376, 732372, 5, 5, 0, 0, 0, 0, 0, 25, datetime.datetime(2006, 3, 2, 0, 0), 31.375, 30.125, 1.3794097900390625, 1.0775715112686157, 0.16599373519420624, 5.387857437133789, 0.5038375854492188, 18.28999900817871, 0.2009933441877365, 17.99799919128418, 0.16647663712501526, 0.1611703485250473, 1.0049667358398438, 89.98999786376953, 0.6369724273681641, 0.2568630576133728),
 (24114, 732371, 732375, 732372, 5, 5, 0, 0, 0, 0, 0, 19, datetime.datetime(2006, 3, 1, 0, 0), 31.625, 30.125, 1.4004859924316406, 1.0846893787384033, 0.17591488361358643, 5.4234466552734375, 0.5505657196044922, 18.32999

In [24]:
from pandas import DataFrame

def select_range(table=mhw_tbl, lat_start=30.0, lat_end=35.0, lon_start=30.0, lon_end=35.0, date_start='Jun 1 2005', date_end='Jun 1 2006'):
  """
  Queries MHW_Events table given ranges (latitude, longitude, date), and returns
  a dataframe containing all the entries of the table which meet that criteria. 

  Query (SQL):
  SELECT * FROM MHW_Events 
  WHERE lat BETWEEN ? AND ? 
  AND lon BETWEEN ? AND ?
  AND date BETWEEN ? AND ?

  :param table:
    SQLAlchemy Table object containing MHW_Events
    Run the following code before calling this function:
      engine = sqlalchemy.create_engine('sqlite:///'+mhw_file)
      metadata = sqlalchemy.MetaData()
      mhw_tbl = sqlalchemy.Table('MHW_Events', metadata, autoload=True, autoload_with=engine)
  
  :param lat_start:
    double representing the lower end of latitude range
  
  :param lat_end:
    double representing the upper end of latitude range
  
  :param lon_start:
    double representing the lower end of longitude range
  
  :param lon_end:
    double representing the lower end of longitude range
  
  :param date_start:
    String representing the start date
    Format: 'Mon Day Year' where Day and Year are integers
  
  :param date_end:
    String representing the start date
    Format: 'Mon Day Year' where Day and Year are integers
  
  :return range_df:
    Pandas dataframe containing query results
  """

  # can't run if table not properly set up
  # if not table:
  #   print("Please load the MHW_Events table into a Table object.")
  #   return None
  
  # set up query
  query = sqlalchemy.select([table]).where(sqlalchemy.and_(
    # range of lat 
    sqlalchemy.between(
        # pass functions?
        table.columns.lat, 30.0, 35.0
    ),
    # and range of lon
    sqlalchemy.between(
        # pass functions?
        table.columns.lon, 30.0, 35.0
    ),
    # and range of dates
    sqlalchemy.between(
        # pass functions?
        table.columns.date, datetime.strptime('Jun 1 2005', '%b %d %Y'), datetime.strptime('Jun 1 2006', '%b %d %Y')
    )
    ))
  
  # execute query
  result = connection.execute(query)

  # get df
  range_df = DataFrame(result.fetchall())
  # range_df.columns = query.keys()

  # return df
  return range_df

In [25]:
df = select_range()
df

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,24022,732152,732156,732154,5,5,0,0,0,0,...,0.462904,27.799999,0.307270,27.644001,0.131095,0.122898,1.536352,138.220001,0.153096,0.225403
1,24023,732372,732376,732372,5,5,0,0,0,0,...,0.503838,18.289999,0.200993,17.997999,0.166477,0.161170,1.004967,89.989998,0.636972,0.256863
2,24114,732371,732375,732372,5,5,0,0,0,0,...,0.550566,18.330000,0.235694,18.016001,0.176122,0.175909,1.178471,90.080002,0.464554,0.124975
3,24204,732371,732376,732372,6,6,0,0,0,0,...,0.680984,18.410000,0.299216,18.023333,0.199818,0.202622,1.795294,108.139999,0.388814,0.271603
4,24293,732370,732375,732372,6,6,0,0,0,0,...,0.649967,18.320000,0.315011,17.984999,0.181617,0.174619,1.890064,107.909996,0.318320,0.149031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,24259,732371,732375,732371,5,5,0,0,0,0,...,0.346935,17.949999,0.128878,17.745998,0.114986,0.108185,0.644390,88.729996,0.520012,0.069498
258,24260,732455,732469,732467,15,14,0,0,0,0,...,1.180807,25.350000,0.483286,24.099998,0.507269,0.765654,7.249289,361.499969,0.112175,0.513492
259,24351,732306,732311,732309,6,6,0,0,0,0,...,0.457418,19.609999,0.298462,19.490000,0.108901,0.174738,1.790771,116.940002,0.149296,0.171850
260,24352,732371,732377,732371,7,7,0,0,0,0,...,0.382647,17.920000,0.216725,17.778574,0.107604,0.104257,1.517073,124.450012,0.557278,0.119052
